## Import Libraries
Import necessary libraries for model building, data loading, and preprocessing.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.api.preprocessing import image_dataset_from_directory
from tensorflow.data import AUTOTUNE, Dataset
from keras.api.applications import MobileNetV2
from keras.api.applications.mobilenet_v2 import preprocess_input
from keras.api.layers import Input, RandomRotation, RandomZoom, RandomFlip, RandomTranslation, GlobalAveragePooling2D, Dense, BatchNormalization, Dropout
from keras.api.models import Model, load_model, save_model
from keras.api.activations import relu, softmax
from keras.api.regularizers import L2
from keras.api.optimizers import Adam
from keras.api.losses import SparseCategoricalCrossentropy
from keras.api.metrics import SparseCategoricalAccuracy
from numpy import ndarray, dtype, float32
from typing import Any, Tuple, List, TypeAlias

Array: TypeAlias = ndarray[Any, dtype[float32]]

## Load Data
Load training and validation datasets.

In [ ]:
height: int = 224
width: int = 224

In [ ]:
def load_data(
        directory: str,
        color: str,
        batch: int,
        size: Tuple[int, int],
        split: float
) -> Tuple[Dataset, Dataset]:
    dataset: List = image_dataset_from_directory(directory = directory, labels = "inferred", label_mode = "int", color_mode = color, batch_size = batch, image_size = size, shuffle = True, validation_split = split, subset = "both", seed = 42)

    return dataset[0], dataset[1]

In [ ]:
train_dataset, test_dataset = load_data("data/Vehicle Colour", "rgb", 32, (height, width), 0.1)

## Data Exploration

### Shape of Images
Check the shape of the images and labels in the dataset.

In [ ]:
X_train: Array = np.array([], dtype = float32)
y_train: Array = np.array([], dtype = float32)

for images, labels in train_dataset.take(1):
    print(f"X_train {images.shape}")
    print(f"y_train {labels.shape}")

    X_train = images[:10]
    y_train = labels[:10].numpy()

for images, labels in test_dataset.take(1):
    print(f"X_test  {images.shape}")
    print(f"y_test  {labels.shape}")

### Display Images
Visualize a few images from the training dataset to ensure correctness.

In [ ]:
plt.figure(figsize = (10, 3.5))
plt.subplots_adjust(wspace = 0.5, hspace = 0.5)

for i in range(X_train.shape[0]):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_train[i] / 255)
    plt.title(y_train[i])

## Preprocess Data
Preprocess the data using Keras `preprocess_input()` function, and then prefetch the data.

In [ ]:
train_dataset = train_dataset.map(lambda
        x,
        y: (preprocess_input(x), y))

test_dataset = test_dataset.map(lambda
        x,
        y: (preprocess_input(x), y))

In [ ]:
train_dataset = train_dataset.prefetch(buffer_size = AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size = AUTOTUNE)

## Build The Model

### Base Model (MobileNetV2)
Use the pre-trained MobileNetV2 model without the top layers.

In [ ]:
base_model: MobileNetV2 = MobileNetV2(input_shape = (height, width, 3), include_top = False, weights = "imagenet")

In [ ]:
base_model.trainable = False

In [ ]:
inputs = Input(shape = (height, width, 3), dtype = float32)

T = RandomRotation((-0.25, 0.25), interpolation = "nearest")(inputs)
T = RandomZoom(height_factor = (-0.1, 0.1), width_factor = (-0.1, 0.1), interpolation = "nearest")(T)
T = RandomFlip(mode = "horizontal_and_vertical")(T)
T = RandomTranslation(height_factor = (-0.1, 0.1), width_factor = (-0.1, 0.1), interpolation = "nearest")(T)
T = base_model(T, training = False)
T = GlobalAveragePooling2D()(T)
T = Dense(units = 64, activation = relu, kernel_regularizer = L2(0.01))(T)
T = BatchNormalization()(T)
T = Dropout(rate = 0.3)(T)

outputs = Dense(units = 3, activation = softmax)(T)

model: Model = Model(inputs, outputs)

In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.001), loss = SparseCategoricalCrossentropy(), metrics = [SparseCategoricalAccuracy()])

### Initial Training
Train the model with the pre-trained base model frozen.

In [ ]:
model.fit(train_dataset, epochs = 1, validation_data = test_dataset)

### Fine-Tuning
Unfreeze the base model for fine-tuning.

In [ ]:
base_model.trainable = True

In [ ]:
model.compile(optimizer = Adam(learning_rate = 0.00001), loss = SparseCategoricalCrossentropy(), metrics = [SparseCategoricalAccuracy()])

### Retrain Model
Retrain the model with the pre-trained base model unfrozen.

In [ ]:
model.fit(train_dataset, epochs = 1, validation_data = test_dataset)

### Load Model
Load the trained model and use it.

In [ ]:
# model: Model = load_model("models/Vehicle Colour.keras")

## Evaluate the Model
Check the model's accuracy on the train and test sets.

In [ ]:
print(model.evaluate(train_dataset))
print(model.evaluate(test_dataset))

## Save the Model
Save the trained model for future use.

In [ ]:
save_model(model = model, filepath = "models/Vehicle Colour.keras", zipped = True, overwrite = True)